<a href="https://colab.research.google.com/github/poojya100/1BM23CS303_BIS_LAB/blob/main/BisLab-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import*
x=symbols('x')
f=sin(x)**2
i=integrate(f,(x,0,pi/2))
display(i)

pi/4

In [1]:
import numpy as np

class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        """
        distances: 2D numpy array of distances between cities
        n_ants: number of ants per iteration
        n_best: number of best ants who deposit pheromone
        n_iterations: number of iterations
        decay: rate of pheromone evaporation (0 < decay < 1)
        alpha: pheromone importance
        beta: distance importance (heuristic)
        """
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
            self.pheromone *=(1 - self.decay)
        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for i in range(len(path)):
            from_city = path[i][0]
            to_city = path[i][1]
            total_dist += self.distances[from_city][to_city]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)  # start from city 0
            dist = self.gen_path_dist(path)
            all_paths.append((path, dist))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for _ in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start))  # return to start
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)
        norm_row = row / row.sum()
        move = np.random.choice(self.all_inds, 1, p=norm_row)[0]
        return move

# Example usage:
if __name__ == "__main__":
    # Distance matrix (symmetric) between 4 cities
    distances = np.array([
        [np.inf, 2, 2, 5],
        [2, np.inf, 3, 4],
        [2, 3, np.inf, 1],
        [5, 4, 1, np.inf]
    ])

    ant_colony = AntColony(distances, n_ants=10, n_best=3, n_iterations=100, decay=0.1, alpha=1, beta=2)
    shortest_path = ant_colony.run()
    print("Shortest path:", shortest_path[0])
    print("Distance:", shortest_path[1])


Shortest path: [(0, np.int64(2)), (np.int64(2), np.int64(3)), (np.int64(3), np.int64(1)), (np.int64(1), 0)]
Distance: 9.0
